In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import subprocess
CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)
if CUDA_version == "10.0":
   torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
   torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
   torch_version_suffix = ""
else:
   torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex
!pip install git+https://github.com/openai/CLIP.git
!pip install ftfy regex tqdm

CUDA version: 11.8
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu110 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2)
ERROR: No matching distribution found for torch=

In [57]:
from pathlib import Path
from torchvision.datasets.utils import download_url
import json
import torch
import clip
import os
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
# directory of input images
Img_dir = "/content/drive/MyDrive/image_data/"

# text of "rabbit" and "face"
text = clip.tokenize(["rabbit", "face"]).to(device)

data_dir = Path(Img_dir)
data_files = data_dir.glob('*.png') #Images by .png name

for image_name in data_files: #for every image in the directory
  image = preprocess(Image.open(image_name)).unsqueeze(0).to(device)
  with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    #calculation of probability
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

  print("image name", image_name)
  print("Probability of [[rabbit, face]]", probs)
  print("")




image name /content/drive/MyDrive/image_data/moon_binary_th90_resize_lat_0N.png
Probability of [[rabbit, face]] [[0.5 0.5]]

image name /content/drive/MyDrive/image_data/moon_binary_th90_resize_lat_25N.png
Probability of [[rabbit, face]] [[0.342 0.658]]

image name /content/drive/MyDrive/image_data/moon_binary_th90_resize_lat_50N.png
Probability of [[rabbit, face]] [[0.137 0.863]]

image name /content/drive/MyDrive/image_data/moon_binary_th70_resize_lat_0N.png
Probability of [[rabbit, face]] [[0.57 0.43]]

image name /content/drive/MyDrive/image_data/moon_binary_th90_resize_blur_kernel5_lat_50N.png
Probability of [[rabbit, face]] [[0.26 0.74]]

image name /content/drive/MyDrive/image_data/moon_binary_th90_resize_blur_kernel5_lat_25N.png
Probability of [[rabbit, face]] [[0.461 0.539]]

image name /content/drive/MyDrive/image_data/moon_binary_th70_resize_lat_25N.png
Probability of [[rabbit, face]] [[0.457 0.543]]

image name /content/drive/MyDrive/image_data/moon_binary_th90_resize_blur_